In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r /content/drive/MyDrive/Lameirao/requirements.txt

import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


nltk.download('stopwords')
nltk.download('punkt')

from unidecode import unidecode
## ler arquivo json em pandas
df = pd.read_json(r'/content/drive/MyDrive/Lameirao/data2.json')

df = df.drop(['id'], axis=1)
## expandir coluna annotations
df = pd.concat([df.drop(['annotations'], axis=1), df['annotations'].apply(pd.Series)], axis=1)
## expandir coluna 0 e renomear para annotations
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna result e renomear para result
df = pd.concat([df.drop(['result'], axis=1), df['result'].apply(pd.Series)], axis=1)
df = df.drop(['id'], axis=1)

## expandir coluna 0 e renomear para result
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna value e renomear para value
df = pd.concat([df.drop(['value'], axis=1), df['value'].apply(pd.Series)], axis=1)
## dropar choices nulos
df = df.dropna(subset=['choices'])
## obter choices 
df['choices'] = df['choices'].apply(lambda x: x[0])

## expandir coluna data
df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1)

df_noticia_original = df.copy()


padrao_data_cepea = r"Cepea, \d{2}/\d{2}/\d{4} - "
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_data_cepea, '', x))

## remover a palavra 'cepea' das noticias
padrao_cepea = r"Cepea"
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_cepea, '', x, flags=re.IGNORECASE))

## remover numeros das noticias
padrao_numeros = r'[0-9]+'
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_numeros, '', x))

## noticia que contem a palavra 'soja'
df = df[df['titulo'].str.contains('soja', flags=re.IGNORECASE)]

## remover noticias com choice 'desclassificar'
df = df[df['choices'] != 'Desclassificar']
df.count()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 37.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=a7b38bc6921422d9fb73af5ec7edac38daaf45a07c045ecbb8b36e856924569f
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
<ipython-input-2-845cf612dab6>:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['result'], axis=1), df['result'].apply(pd.Series)], axis=1)


file_upload                 701
drafts                      701
predictions                 701
meta                        701
created_at                  701
updated_at                  701
inner_id                    701
total_annotations           701
cancelled_annotations       701
total_predictions           701
comment_count               701
unresolved_comment_count    701
last_comment_updated_at       0
project                     701
updated_by                  701
comment_authors             701
completed_by                701
was_cancelled               701
ground_truth                701
created_at                  701
updated_at                  701
lead_time                   701
prediction                  701
result_count                701
unique_id                   701
last_action                   0
task                        701
project                     701
updated_by                  701
parent_prediction             0
parent_annotation             0
last_cre

In [ ]:
# Selecionar apenas as colunas necessárias
columns_to_select = ['id', 'data', 'noticia', 'titulo', 'choices', 'unique_id']

df = df[columns_to_select]
df.dropna(subset=['noticia'])

# Pré-processamento dos dados
stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):
    # remover acentuação
    text = unidecode(text)
    # Remover pontuações
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenização
    words = word_tokenize(text.lower())
    # Remover stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

#df['noticia'] = df['noticia'].apply(preprocess_text)

In [ ]:
import time

def calculate_eta(start_time, current_progress, total_progress):
    elapsed_time = time.time() - start_time
    progress_percentage = current_progress / total_progress
    estimated_total_time = elapsed_time / progress_percentage
    estimated_remaining_time = estimated_total_time - elapsed_time
    return estimated_remaining_time

In [ ]:
import nlpaug.augmenter.word as naw
import pandas as pd

augmenter = naw.ContextualWordEmbsAug()

augmented_dataset = []

start_time = time.time()
total_progress = len(df)

count = 0
for index, row in df.iterrows():
    augmented_text = augmenter.augment(row['noticia'])
    augmented_news = {'noticia': augmented_text, 'choices': row['choices']}
    augmented_dataset.append(augmented_news)
    
    count += 1 
    eta = calculate_eta(start_time, count, total_progress)
    print(f"Progress: {count}/{total_progress} | ETA: {eta:.2f} seconds")

augmented_df = pd.DataFrame(augmented_dataset)

augmented_df.to_json('/content/drive/MyDrive/Lameirao/export_dataframe.json')

In [ ]:
df2 = pd.read_json(r'/content/drive/MyDrive/Lameirao/export_dataframe.json')

df2 = df2.explode('noticia')
df2['choices'] = df2['choices'].ffill()

In [ ]:
df_treino = df[:int(len(df) * 0.8)]
df_validacao = df[int(len(df) * 0.8):]

#df_treino = df2[:int(len(df) * 0.8)]
#df_validacao = df2[int(len(df) * 0.8):]

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import TrainingArguments

import numpy as np

from typing import List, Optional, Tuple, Union

# Carregar o modelo pré-treinado BERTimbau
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=3)
model = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

modelo_soja = model

modelo_soja

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29794, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
class LIABertClassifier(nn.Module):
    def __init__(self,model,num_labels):
        super(LIABertClassifier,self).__init__()
        self.bert = model
        self.config = model.config
        self.num_labels = num_labels
        self.cls = nn.Linear(self.config.hidden_size,200)
        self.dropout = nn.Dropout(p=0.5)
        self.cls2 = nn.Linear(768,num_labels)

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        ) ->Tuple[torch.Tensor]:

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        sequence_output = outputs[0][:,0,:]
        prediction = self.dropout(sequence_output)
        prediction = self.cls2(prediction)
        return prediction
        

In [ ]:
torch.cuda.empty_cache()

from torch.utils.data import DataLoader
from transformers import AdamW, get_scheduler
import sklearn.model_selection as model_selection
from torch.optim.lr_scheduler import OneCycleLR

# Map string labels to integer labels
label_map = {'Negativa': 0, 'Positiva': 1, 'Neutra': 2}
ytrain_global = np.array([label_map[label] for label in df_treino['choices']])
xtrain_global = np.array(df_treino['noticia'])

xtrain, xval, ytrain, yval = model_selection.train_test_split(xtrain_global, ytrain_global, test_size=0.20, random_state=42,shuffle=True)

print(f"xtrain: {len(xtrain)} - xval: {len(xval)} - ytrain: {len(ytrain)} - yval: {len(yval)}")

train_encodings = tokenizer(xtrain.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(xval.tolist(), truncation=True, padding=True,max_length=512, return_tensors='pt')

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        label = torch.tensor(self.labels[idx])
        return (item,label)

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_encodings, ytrain)
val_dataset = MyDataset(val_encodings, yval)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LIABertClassifier(modelo_soja, 3)

model.to(device)

dl_train = DataLoader(train_dataset,batch_size=3)
dl_eval  = DataLoader(val_dataset,batch_size=3)
x,y = next(iter(dl_train))

batch = {k: v.to(device) for k, v in x.items()}
model.to(device)


xtrain: 448 - xval: 112 - ytrain: 448 - yval: 112


LIABertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optim = torch.optim.AdamW(optimizer_grouped_parameters, lr=2e-5, eps = 1e-08)

loss_fct = nn.CrossEntropyLoss()

num_epochs = 20

from transformers import get_scheduler

scheduler = get_scheduler(
    "linear",
    optim,
    num_warmup_steps=0,
    num_training_steps=num_epochs*len(dl_train)
)

# scheduler = OneCycleLR(optim, max_lr=max_lr, total_steps=total_steps,
#                        div_factor=div_factor, pct_start=pct_start)

for epoch in range(num_epochs):
    lepochs = []
    for batch,y in dl_train:
        model.train()
        batch = {k: v.to(device) for k, v in batch.items()}
        y = y.to(device)
        outputs = model(**batch)
        loss = loss_fct(outputs,y)
        lepochs.append(loss.cpu().item())
        loss.backward()
        optim.step()
        scheduler.step()
        optim.zero_grad()
        current_lr = optim.param_groups[0]['lr']

        print(f"Epoch {epoch}, Loss={loss:.4f}, LR={current_lr:.6f}")

Epoch 0, Loss=1.1940, LR=0.000020
Epoch 0, Loss=1.2689, LR=0.000020
Epoch 0, Loss=0.9555, LR=0.000020
Epoch 0, Loss=0.9986, LR=0.000020
Epoch 0, Loss=1.1674, LR=0.000020
Epoch 0, Loss=1.3287, LR=0.000020
Epoch 0, Loss=0.5005, LR=0.000020
Epoch 0, Loss=1.3221, LR=0.000020
Epoch 0, Loss=1.4100, LR=0.000020
Epoch 0, Loss=0.9575, LR=0.000020
Epoch 0, Loss=1.5714, LR=0.000020
Epoch 0, Loss=1.1284, LR=0.000020
Epoch 0, Loss=1.0045, LR=0.000020
Epoch 0, Loss=1.5028, LR=0.000020
Epoch 0, Loss=1.2421, LR=0.000020
Epoch 0, Loss=1.2521, LR=0.000020
Epoch 0, Loss=1.6071, LR=0.000020
Epoch 0, Loss=0.8806, LR=0.000020
Epoch 0, Loss=1.1318, LR=0.000020
Epoch 0, Loss=1.3631, LR=0.000020
Epoch 0, Loss=1.0417, LR=0.000020
Epoch 0, Loss=0.9897, LR=0.000020
Epoch 0, Loss=1.2720, LR=0.000020
Epoch 0, Loss=1.0591, LR=0.000020
Epoch 0, Loss=1.0435, LR=0.000020
Epoch 0, Loss=1.1733, LR=0.000020
Epoch 0, Loss=1.4123, LR=0.000020
Epoch 0, Loss=1.2552, LR=0.000020
Epoch 0, Loss=1.2729, LR=0.000020
Epoch 0, Loss=

In [ ]:
model.eval()
ytrue = []
ypred = []
for batch,y in dl_eval:
    batch = {k: v.to(device) for k, v in batch.items()}
    y = y.to(device)
    with torch.no_grad():
        outputs = model(**batch)
        ytrue.extend(y.cpu().numpy().tolist())
        ypred.extend(outputs.argmax(axis=1).cpu().numpy().tolist())

print(classification_report(ytrue,ypred))

              precision    recall  f1-score   support

           0       0.69      0.86      0.77        36
           1       0.87      0.85      0.86        46
           2       0.64      0.47      0.54        30

    accuracy                           0.75       112
   macro avg       0.73      0.73      0.72       112
weighted avg       0.75      0.75      0.74       112



In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Lameirao/modelor_soja.pt')

In [ ]:
#torch.save(model.state_dict(), '/content/drive/MyDrive/modelo_soja.pt')

## predict from string
frase = """
O preço da soja em Paranaguá (PR) em abril registrou o maior aumento médio mensal desde agosto de 2020,
informa o Centro de Estudos Avançados em Economia Aplicada (Cepea), da Esalq/USP.
Considerando a inflação, a média de R$ 145,24 por saca de 60 quilos seria a melhor desde março de 2020.
"""
frase = preprocess_text(frase)
frase = tokenizer(frase, truncation=True, padding=True, max_length=512, return_tensors='pt')
frase = {k: v.to(device) for k, v in frase.items()}
model.eval()
with torch.no_grad():
    outputs = model(**frase)
    if outputs.argmax(axis=1).cpu().numpy().tolist()[0] == 0:
        print('Negativa')
    elif outputs.argmax(axis=1).cpu().numpy().tolist()[0] == 1:
        print('Positiva')
    else:
        print('Neutra')
    print(outputs.argmax(axis=1).cpu().numpy().tolist())

Positiva
[1]


In [ ]:
# Selecionando apenas notícias positivas
df_positivas = df_validacao[df_validacao['choices'] == 'Positiva'].head(30)

# Selecionando apenas notícias negativas
df_negativas = df_validacao[df_validacao['choices'] == 'Negativa'].head(30)

# Selecionando apenas notícias neutras
df_neutras = df_validacao[df_validacao['choices'] == 'Neutra'].head(30)

df_concatenado = pd.concat([df_positivas, df_negativas, df_neutras])

# predict
df_concatenado['noticia'] = df_concatenado['noticia'].apply(preprocess_text)
df_concatenado['noticia'] = df_concatenado['noticia'].apply(lambda x: re.sub(padrao_data_cepea, '', x))
df_concatenado['noticia'] = df_concatenado['noticia'].apply(lambda x: re.sub(padrao_cepea, '', x, flags=re.IGNORECASE))
df_concatenado['noticia'] = df_concatenado['noticia'].apply(lambda x: re.sub(padrao_numeros, '', x))

xtest = np.array(df_concatenado['noticia'])
ytest = np.array([label_map[label] for label in df_concatenado['choices']])
test_encodings = tokenizer(xtest.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
test_dataset = MyDataset(test_encodings, ytest)
dl_test = DataLoader(test_dataset,batch_size=8)

model.eval()
ytrue = []
ypred = []
for batch,y in dl_test:
    batch = {k: v.to(device) for k, v in batch.items()}
    y = y.to(device)
    with torch.no_grad():
        outputs = model(**batch)
        ytrue.extend(y.cpu().numpy().tolist())
        ypred.extend(outputs.argmax(axis=1).cpu().numpy().tolist())

print(classification_report(ytrue,ypred))

              precision    recall  f1-score   support

           0       0.77      0.90      0.83        30
           1       0.79      0.87      0.83        30
           2       0.77      0.57      0.65        30

    accuracy                           0.78        90
   macro avg       0.78      0.78      0.77        90
weighted avg       0.78      0.78      0.77        90

